In [2]:
import pandas as pd
import numpy as np
import psycopg2
import json
from sqlalchemy import create_engine


## this is a _Data_Engineer_interview

In [3]:
df=pd.read_csv("C:/etl/etl_data_interview/candidates.csv",sep=";")
print(df.head())

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

# Analisis Exploratorio De Los Datos
## ¿Qué tipo de dato son las variables del conjunto de datos?

In [34]:
df.dtypes

First Name                   object
Last Name                    object
Email                        object
Application Date             object
Country                      object
YOE                           int64
Seniority                    object
Technology                   object
Code Challenge Score          int64
Technical Interview Score     int64
dtype: object

## ¿Cuántas variables de cada tipo de dato tenemos en el conjunto de datos?

In [4]:
(
    df
    .dtypes
    .value_counts()
)

object    7
int64     3
dtype: int64

## ¿Cuántas variables y observaciones tenemos en el conjunto de datos?

In [7]:
df.shape

(50000, 10)

## ¿Existen valores nulos explicitos en el conjunto de datos?

In [5]:
(   df
    .isnull()
    .any()
)

First Name                   False
Last Name                    False
Email                        False
Application Date             False
Country                      False
YOE                          False
Seniority                    False
Technology                   False
Code Challenge Score         False
Technical Interview Score    False
dtype: bool

In [6]:
df.describe(include='all')

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
count,50000,50000,50000,50000,50000,50000.000000,50000,50000,50000.000000,50000.000000
unique,3007,474,49833,1646,244,NaN,7,24,NaN,NaN
top,Sarai,Murazik,fern70@gmail.com,2020-07-07,Malawi,NaN,Intern,Game Development,NaN,NaN
freq,33,138,3,50,242,NaN,7255,3818,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,15.286980,NaN,NaN,4.996400,5.003880
std,NaN,NaN,NaN,NaN,NaN,8.830652,NaN,NaN,3.166896,3.165082
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,2.000000,2.000000
50%,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,5.000000,5.000000
75%,NaN,NaN,NaN,NaN,NaN,23.000000,NaN,NaN,8.000000,8.000000


## Solo las numéricas

In [32]:
df.describe(include=[np.number])

,YOE,Code_Challenge_Score,Technical_Interview_Score
count,50000.000000,50000.000000,50000.000000
mean,15.286980,4.996400,5.003880
std,8.830652,3.166896,3.165082
min,0.000000,0.000000,0.000000
25%,8.000000,2.000000,2.000000
50%,15.000000,5.000000,5.000000
75%,23.000000,8.000000,8.000000
max,30.000000,10.000000,10.000000


## Solo categóricas - 1

In [33]:
df.describe(include=object)

,First_Name,Last_Name,Email,Application_Date,Country,Seniority,Technology
count,50000,50000,50000,50000,50000,50000,50000
unique,3007,474,49833,1646,244,7,24
top,Sarai,Murazik,fern70@gmail.com,2020-07-07,Malawi,Intern,Game Development
freq,33,138,3,50,242,7255,3818


## checking the different values of the column Technology to see which one to group together

In [8]:
unique_values = df['Technology'].unique()
print(unique_values)

['Data Engineer' 'Client Success' 'QA Manual'
 'Social Media Community Management' 'Adobe Experience Manager' 'Sales'
 'Mulesoft' 'DevOps' 'Development - CMS Backend' 'Salesforce'
 'System Administration' 'Security' 'Game Development'
 'Development - CMS Frontend' 'Security Compliance'
 'Development - Backend' 'Design'
 'Business Analytics / Project Management' 'Development - Frontend'
 'Development - FullStack' 'Business Intelligence'
 'Database Administration' 'QA Automation' 'Technical Writing']


## checking the count of different values of the column Technology

In [8]:
conteo_variables = df['Technology'].value_counts()
conteo_variables

Game Development                           3818
DevOps                                     3808
Social Media Community Management          2028
System Administration                      2014
Mulesoft                                   1973
Development - Backend                      1965
Development - FullStack                    1961
Adobe Experience Manager                   1954
Data Engineer                              1951
Security                                   1936
Development - CMS Frontend                 1934
Business Intelligence                      1934
Database Administration                    1933
Client Success                             1927
Design                                     1906
QA Manual                                  1902
Technical Writing                          1901
QA Automation                              1892
Sales                                      1890
Development - Frontend                     1887
Development - CMS Backend               

# TRANSFORM 

In [3]:
new_Columns_names=["First_Name","Last_Name","Email","Application_Date","Country","YOE","Seniority","Technology","Code_Challenge_Score","Technical_Interview_Score"]
df.columns=new_Columns_names
df

,First_Name,Last_Name,Email,Application_Date,Country,YOE,Seniority,Technology,Code_Challenge_Score,Technical_Interview_Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


## filter the year for analize the hireds for each one

In [104]:
dfpa = df['Application_Date'].str[:4]
a=dfpa.value_counts()

print(a)

2020    11237
2018    11061
2021    11051
2019    11009
2022     5642
Name: Application_Date, dtype: int64


the year 2022 suspiciously has fewer records

## Calculate the count of unique months for each year.

In [4]:
df_to_calculate=df.copy()

df_months_counts = pd.to_datetime(df_to_calculate['Application_Date'])


df_to_calculate['Year'] = df_months_counts.dt.year
df_to_calculate['Month'] = df_months_counts.dt.month

months_counts_by_year = df_to_calculate.groupby('Year')['Month'].nunique()

print(months_counts_by_year)

Year
2018    12
2019    12
2020    12
2021    12
2022     7
Name: Month, dtype: int64


It seems that the data are incomplete for the year 2022 and the months go up to July, that is to say, the graphs made from 2022 onwards will be wrong, indicating that the complete analysis of the whole year is not available.

## Adding the hired column 

### if was hired we assign 1 in this column if is not 0

In [5]:
def was_hired(Code_Challenge_Score,Technical_Interview_Score):
    if Code_Challenge_Score >= 7 and  Technical_Interview_Score >= 7:
        return 1
    else:
        return 0

In [6]:
df['Hired'] = df.apply(lambda row: was_hired(row['Code_Challenge_Score'], row['Technical_Interview_Score']), axis=1)
df

,First_Name,Last_Name,Email,Application_Date,Country,YOE,Seniority,Technology,Code_Challenge_Score,Technical_Interview_Score,Hired
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,0
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,0
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,1
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,0
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,1
...,...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1,0
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2,0
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1,0
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0,0


In [4]:
# Lee las credenciales desde el archivo JSON
with open('db_config.json', 'r') as file:
    credenciales = json.load(file)

## Connection with the Database

In [6]:
# Parámetros de conexión
db_params = {
    "host": credenciales["host"],
    "port": credenciales["port"],
    "database": credenciales["database"],
    "user": credenciales["user"],
    "password": credenciales["password"]
}

# Intentar conectarse a la base de datos
try:
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    # Ejecutar consultas aquí
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Conexión exitosa a PostgreSQL:", version)

    

    # Consultar el catálogo information_schema para obtener las tablas
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    tables = cursor.fetchall()

    # Imprimir las tablas
    print("Tablas en la base de datos:")
    for table in tables:
        print(table[0])
        
    table_to_delete = "Applicants"
    #cursor.execute("DROP TABLE Applicants_def")
   

 


    # Confirmar los cambios
    connection.commit()

    # Cerrar la conexión
    cursor.close()
    connection.close()

except Exception as e:
    print("Error de conexión:", e)

Conexión exitosa a PostgreSQL: ('PostgreSQL 15.3 (Debian 15.3-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
Tablas en la base de datos:
Applicants_def
Applicants_interview_ready
applicants_interview_ready
applicants_interview_good


In [8]:
# Creo que se puede mejorar, llamando solamente al connection
connection= psycopg2.connect(**db_params)
cursor = connection.cursor()

create_table_query = """
    CREATE TABLE IF NOT EXISTS applicants_interview_good (
    id serial PRIMARY KEY,
    first_name varchar,
    last_name varchar,
    email varchar,
    application_date date,
    country varchar,
    yoe integer,
    seniority varchar,
    technology varchar,
    code_challenge_score integer,
    technical_interview_score integer,
    Hired integer    
);
"""
cursor.execute(create_table_query)
connection.commit()


#cursor.close()

In [9]:
# Crear una cadena de conexión a la base de datos PostgreSQL
db_connection_string = (
    f"postgresql://{db_params['user']}:{db_params['password']}@"
    f"{db_params['host']}:{db_params['port']}/{db_params['database']}"
)

# Crear una instancia de SQLAlchemy Engine
engine = create_engine(db_connection_string)

# Cargar el DataFrame en la tabla "mi_tabla" en la base de datos
tabla_destino = "applicants_interview_good"
df.to_sql(tabla_destino, engine, if_exists="replace", index=False)

# Cierra la conexión a la base de datos
engine.dispose()

## Verificamos que este los datos en la BD

In [10]:
connection= psycopg2.connect(**db_params)
cursor = connection.cursor()
consulta = "SELECT * FROM applicants_interview_good"

cursor.execute(consulta)
#Obtener los resultados de la consulta
resultados = cursor.fetchall()
for fila in resultados:
    print(fila)
cursor.close()

('Bernadette', 'Langworth', 'leonard91@yahoo.com', '2021-02-26', 'Norway', 2, 'Intern', 'Data Engineer', 3, 3, 0)
('Camryn', 'Reynolds', 'zelda56@hotmail.com', '2021-09-09', 'Panama', 10, 'Intern', 'Data Engineer', 2, 10, 0)
('Larue', 'Spinka', 'okey_schultz41@gmail.com', '2020-04-14', 'Belarus', 4, 'Mid-Level', 'Client Success', 10, 9, 1)
('Arch', 'Spinka', 'elvera_kulas@yahoo.com', '2020-10-01', 'Eritrea', 25, 'Trainee', 'QA Manual', 7, 1, 0)
('Larue', 'Altenwerth', 'minnie.gislason@gmail.com', '2020-05-20', 'Myanmar', 13, 'Mid-Level', 'Social Media Community Management', 9, 7, 1)
('Alec', 'Abbott', 'juanita_hansen@gmail.com', '2019-08-17', 'Zimbabwe', 8, 'Junior', 'Adobe Experience Manager', 2, 9, 0)
('Allison', 'Jacobs', 'alba_rolfson27@yahoo.com', '2018-05-18', 'Wallis and Futuna', 19, 'Trainee', 'Sales', 2, 9, 0)
('Nya', 'Skiles', 'madisen.zulauf@gmail.com', '2021-12-09', 'Myanmar', 1, 'Lead', 'Mulesoft', 2, 5, 0)
('Mose', 'Lakin', 'dale_murazik@hotmail.com', '2018-03-13', 'Italy

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

